In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
exchange = 'bybit'
user = 'your_user_name'
settings = load_settings(exchange, user)
s = 'BTCUSD'
n_days = 7

In [ ]:
#load cache if any
#agg_trades = pd.read_csv('btcusdt_agg_trades_bybit_14_days_2021-01-03.csv').set_index('trade_id')

In [ ]:
agg_trades = await load_trades(exchange, user, s, n_days)

In [ ]:
adf = agg_trades

In [ ]:
adf = adf[adf.price != adf.price.shift(1)]
len(adf), 'n_days', (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24

In [ ]:
# look at jackrabbit results
rdf = pd.concat([pd.read_csv('jackrabbit_results_grid/2021-01-03T14:05:24.csv')])
rdf.columns = ['ema_spans', 'ema_spread', 'entry_qty_equity_multiplier', 'entry_qty_scaling_factor',
               'grid_spacing', 'leverage', 'markup'] + list(rdf.columns[7:])
rdfs = rdf.sort_values('pnl_sum', ascending=[False])
rdfs.head(20)


In [ ]:
settings = {
    'ema_spans': [37328.0, 51671.0],
    'ema_spread': -0.000468,
    'entry_qty_equity_multiplier': 0.00192,
    'entry_qty_scaling_factor': 0.413,
    'grid_spacing': 0.0047,
    'initial_equity': 0.001,
    'leverage': 84.0,
    'maker_fee': -0.00025,
    'markup': 0.00723,
    'min_qty': 1.0,
    'price_step': 0.5,
    'qty_step': 1.0,
    'symbol': 'BTCUSD'
}

In [ ]:
# set best jackrabbit result as settings
'''
bs = dict(rdfs.iloc[0])
for k in bs:
    if k in settings:
        if type(bs[k]) == str:
            settings[k] = eval(bs[k].strip())
        else:
            settings[k] = round(bs[k], 10)
        print(k, settings[k])
settings
'''

In [ ]:
def plot_tdf_(df_, tdf_, side_: int = 0):
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type == 'entry']
        lc = longs[longs.type == 'close']
        ll = longs[longs.type == 'liq']
        le.price.plot(style='b.')
        le.pos_price.plot(style='r-')
        lc.price.plot(style='ro')
        ll.price.plot(style='gx')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type == 'entry']
        sc = shrts[shrts.type == 'close']
        sl = shrts[shrts.type == 'liq']
        se.price.plot(style='r.')
        se.pos_price.plot(style='b-')
        sc.price.plot(style='bo')
        sl.price.plot(style='gx')

In [ ]:
df = prep_df(adf, settings)
print(len(df), len(df) / len(adf))
df[['price', 'bid_thr', 'ask_thr']].iloc[::100].plot()

In [ ]:
trades = backtest(df, settings)

In [ ]:
tdf = pd.DataFrame(trades).set_index('trade_id')
longs = tdf[tdf.side == 'long']
shrts = tdf[tdf.side == 'shrt']
le = longs[longs.type == 'entry']
lc = longs[longs.type == 'close']
ll = longs[longs.type == 'liq']
se = shrts[shrts.type == 'entry']
sc = shrts[shrts.type == 'close']
sl = shrts[shrts.type == 'liq']

margin_max = tdf.margin_cost.max()
pnl_sum = tdf.pnl.sum()
gain = (settings['initial_equity'] + pnl_sum) / settings['initial_equity']
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
average_daily_gain = gain ** (1 / n_days)
liqs = tdf[tdf.type == 'liq']
closes = tdf[tdf.type == 'close']
print('margin_max', margin_max)
print('pnl_sum', pnl_sum)
print('gain', gain)
print('n_days', n_days)
print('average_daily_gain', average_daily_gain)
print('n trades', len(tdf))
print('n closes', len(closes))
print('n liqs', len(liqs))

In [ ]:
tdf.head(40)

In [ ]:
tdf.qty.plot()

In [ ]:
tdf.equity_available.plot()

In [ ]:
tdf.pnl.cumsum().plot()

In [ ]:
liqs

In [ ]:
# inspect liquidations
liq_i = 0
liq_iloc = tdf.index.get_loc(liqs.index[liq_i])
iminus = 10
iplus = 10
tdfc = tdf.iloc[max(0, liq_iloc-iminus):min(liq_iloc+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc, 1 if liqs.iloc[liq_i].side == 'long' else -1)

In [ ]:
# inspect biggest orders
big_i = 0
big_iloc = tdf.index.get_loc(tdf.qty.abs().sort_values().iloc[-big_i])
iminus = 100
iplus = 100
tdfc = tdf.iloc[max(0, big_iloc-iminus):min(big_iloc+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)#, 1 if tdf.iloc[liq_i].side == 'long' else -1)

In [ ]:
plot_tdf_(df, tdf)